    # Web-Scraping

In [2]:
### extract all soccer data

# code für die player stats - scraped nur eine tabelle, wie im alten code

from dotenv import load_dotenv
load_dotenv(override=True)
import os
from os.path import join
import time

from web_scraping import get_soccer_leagues_ids, get_soccer_league_season_standings, get_soccer_teams_url, get_player_season_stats2

current_season = 2022
soccer_league_ids = get_soccer_leagues_ids()

for year in range(1999,2000):
    print(year)
    try:
        for soccer_league_id in soccer_league_ids:
            if soccer_league_id["league_name"] == "Fußball-Bundesliga" or soccer_league_id["league_name"] == "Big 5 European Leagues Combined":
                continue

            # get player stats
            df = get_player_season_stats2(year, current_season, soccer_league_id["league_id"], soccer_league_id["league_stats_name"])
            df.to_csv(join(os.environ["SportsTables"], "soccer", f"{soccer_league_id['league_name'].replace(' ','-')}_player_stats_{year}.csv"), index=False)

            time.sleep(3)
    except:
        print("not possible")
        continue


1999


In [2]:
# zeigt den Fehler, wenn alle Tabellen von den player stats geladen werden sollen

from web_scraping import get_player_season_stats, get_soccer_leagues_ids


current_season = 2022
soccer_league_ids = get_soccer_leagues_ids()

for year in range(1982, 2000):
    for soccer_league_id in soccer_league_ids:
        try:
            standings = get_player_season_stats(year, current_season, soccer_league_id["league_id"], soccer_league_id["league_stats_name"])
        except Exception as e:
            if hasattr(e, 'message'):
                print(e.message)
            else:
                print ('not possible')
                print(e)

        continue


not possible
HTTPConnectionPool(host='localhost', port=52995): Max retries exceeded with url: /session/285905d3b3d58b92b650e03cf6477173/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000227A56E65B0>: Failed to establish a new connection: [WinError 10061] Es konnte keine Verbindung hergestellt werden, da der Zielcomputer die Verbindung verweigerte'))
not possible
'NoneType' object is not callable
not possible
'NoneType' object is not callable
not possible
'NoneType' object is not callable
not possible
'NoneType' object is not callable
not possible
HTTPConnectionPool(host='localhost', port=52995): Max retries exceeded with url: /session/285905d3b3d58b92b650e03cf6477173/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000227A703C9A0>: Failed to establish a new connection: [WinError 10061] Es konnte keine Verbindung hergestellt werden, da der Zielcomputer die Verbindung verweigerte'))
not possible
HTTPConnectionPool(ho

KeyboardInterrupt: 

In [1]:
# zeigt wie die player stats gescraped werden (nur premier league), also alle tabellen und dann alle Tabellen mit dem gleichen Namen werden zusammengefügt

from dotenv import load_dotenv
load_dotenv(override=True)
import os
from os.path import join
import time

from web_scraping import get_soccer_leagues_ids, get_soccer_league_season_standings, get_soccer_teams_url, get_player_season_stats

current_season = 2022
soccer_league_ids = get_soccer_leagues_ids()

for year in range(1999,2000):
    print(year)
    try:
        for soccer_league_id in soccer_league_ids:
            if soccer_league_id["league_name"] == "Fußball-Bundesliga" or soccer_league_id["league_name"] == "Big 5 European Leagues Combined":
                continue

            # get player stats
            df = get_player_season_stats(year, current_season, soccer_league_id["league_id"], soccer_league_id["league_stats_name"])
            df.to_csv(join(os.environ["SportsTables"], "soccer", f"{soccer_league_id['league_name'].replace(' ','-')}_player_stats_{year}.csv"), index=False)

            time.sleep(3)
    except:
        print("not possible")
        pass

1999
Saved Premier-League-Stats_stats_standard_9_1999-2000.csv
Saved Premier-League-Stats_matchlogs_for_1999-2000.csv
Saved Premier-League-Stats_stats_keeper_9_1999-2000.csv
Saved Premier-League-Stats_stats_shooting_9_1999-2000.csv
Saved Premier-League-Stats_stats_playing_time_9_1999-2000.csv
Saved Premier-League-Stats_stats_misc_9_1999-2000.csv
Saved Premier-League-Stats_results1999-200091_overall_1999-2000.csv
Saved Premier-League-Stats_results1999-200091_home_away_1999-2000.csv
not possible


In [ ]:
# scraped all table data without getting httpconnectionpool error
# 464 tables
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time

leagues = {
    'premier_league': list(range(1982, 2000)),
    'ligue_1': list(range(1989, 2000)),
    'serie_a': list(range(1994, 2000)),
    'la_liga': list(range(1993, 2000)),
    'fussball-bundesliga': list(range(1988, 2000)),
    'champions-league': list(range(1992, 2000)),

}

options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome(options=options)

for league, years in leagues.items():
    for year in years:
        if league == 'premier_league':
            url = f'https://fbref.com/en/comps/9/{year}-{year+1}/{year}-{year+1}-Premier-League-Stats'
        elif league == 'ligue_1':
            url = f'https://fbref.com/en/comps/13/{year}-{year+1}/{year}-{year+1}-Ligue-1-Stats'
        elif league == 'serie_a':
            url = f'https://fbref.com/en/comps/11/{year}-{year+1}/{year}-{year+1}-Serie-A-Stats'
        elif league == 'la_liga':
            url = f'https://fbref.com/en/comps/12/{year}-{year+1}/{year}-{year+1}-La-Liga-Stats'
        elif league == 'fussball-bundesliga':
            url = f'https://fbref.com/en/comps/20/{year}-{year+1}/{year}-{year+1}-Bundesliga-Stats'
        elif league == 'champions-league':
            url = f'https://fbref.com/en/comps/8/{year}-{year+1}/{year}-{year+1}-Champions-League-Stats'

        driver.get(url)
        time.sleep(5)
        try:
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            tables = soup.find_all('table')
            for table in tables:
                table_id = table.get('id')
                if table_id is not None:
                    df = pd.read_html(str(table))[0]
                    name = f'{league}_{table_id}_{year}.csv'
                    df.to_csv(name, index=False)
                    print(f'Saved {name}')

        except:
            print("not possible")
            pass

driver.quit()

In [ ]:
# scraped extra tables without getting http error

from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time

leagues = {
    'premier_league': list(range(1982, 2000)),
    'ligue_1': list(range(1989, 2000)),
    'serie_a': list(range(1994, 2000)),
    'la_liga': list(range(1993, 2000)),
    'fussball-bundesliga': list(range(1988, 2000)),
    'champions-league': list(range(1992, 2000)),

}

options = webdriver.ChromeOptions()
options.add_argument('headless')

driver = webdriver.Chrome(options=options)

for league, years in leagues.items():
    for year in years:
        if league == 'premier_league':
            url = f'https://fbref.com/en/comps/9/{year}-{year+1}/schedule/{year}-{year+1}-Premier--Scores-and-Fixtures'
        elif league == 'ligue_1':
            url = f'https://fbref.com/en/comps/13/{year}-{year+1}/schedule/{year}-{year+1}-Ligue-1-Scores-and-Fixtures'
        elif league == 'serie_a':
            url = f'https://fbref.com/en/comps/11/{year}-{year+1}/schedule/{year}-{year+1}-Serie-A-Scores-and-Fixtures'
        elif league == 'la_liga':
            url = f'https://fbref.com/en/comps/12/{year}-{year+1}/schedule/{year}-{year+1}-La-Liga-Scores-and-Fixtures'
        elif league == 'fussball-bundesliga':
            url = f'https://fbref.com/en/comps/20/{year}-{year+1}/schedule/{year}-{year+1}-Bundesliga-Scores-and-Fixtures'

        driver.get(url)
        time.sleep(5)
        try:
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")
            tables = soup.find_all('table')
            for table in tables:
                table_id = table.get('id')
                if table_id is not None:
                    df = pd.read_html(str(table))[0]
                    name = f'{league}_{table_id}_{year}.csv'
                    df.to_csv(name, index=False)
                    print(f'Saved {name}')

        except:
            print("not possible")
            pass

driver.quit()

# Extract column headers from different tables

In [ ]:
import json
from glob import glob
from dotenv import load_dotenv
load_dotenv(override=True)

import os
from os.path import join
import sys
import pandas as pd

metadata = {}

tables = ["player_stats", "season_standings"]

for table in tables:
    column_names = []
    for found_table in glob(join(os.environ["SportsTables"], "soccer", f"*{table}_*.csv")):
        column_names.extend(list(pd.read_csv(found_table).columns))
    
    column_names = list(set(column_names))
    metadata[table] = {}
    for column_name in column_names:
        metadata[table][column_name] = None
    
if os.path.isfile("metadata.json"):
    print("metadata.json exists already!!! ")
else:
    with open("metadata.json", "w") as f:
        json.dump(metadata, f)

# Extract valid header file

In [ ]:
import json
from glob import glob
import os 
#os.environ["SPORTS_DB"] = "/ext/daten-wi/slangenecker/SportsTables"
os.environ["SPORTS_DB"] = "C:/Users/jwuel/DataspellProjects/SportsTables"
from os.path import join
import pandas as pd

def get_sportsDB_soccer_type_mappings():
    with open(join(os.environ["SPORTS_DB"], "Soccer", "Soccer-Relations.json"), "r") as f:
        type_mappings = json.load(f)["soccer-big5Leagues-Players-2021-2022.csv"]["columns"][0]
        return type_mappings


def get_all_sportsDB_soccer_tables(only_file_names:bool=False):
    result = glob(join(os.environ["SPORTS_DB"], "Soccer", "soccerPlayerScraping", "*.csv"))
    if only_file_names:
        return [table.split("/")[-1] for table in result]
    else:
        return result

def load_sportsDB_soccer_table(table_name:str, only_headers:bool=False):
    df = pd.read_csv(join(os.environ["SPORTS_DB"], "Soccer", "soccerPlayerScraping", table_name))
    if only_headers:
        return df.columns
    else:
        return df

In [ ]:
col_name_to_sem_type = get_sportsDB_soccer_type_mappings()

results = {}
for table in get_all_sportsDB_soccer_tables(True):
    print(table)
    results[table.split(".csv")[0]] = {}
    col_names = load_sportsDB_soccer_table(table, True)
    for col_num, col_name in enumerate(col_names):
        try:
            results[table.split(".csv")[0]][f"column_{col_num}"] = { "semanticType": col_name_to_sem_type[col_name]}
        except:
            print(col_name)
    
with open(join(os.environ["SPORTS_DB"], "Soccer", "sportsDB_type_sportsDB.json"), "w") as f:
    json.dump(results, f)